# Heuristics
___

In [20]:
import numpy as np

## Marouane Heuristic 

In [21]:
def johnsons_rule(machine1, machine2):
    # Sort jobs based on min(AM1_i, AM2_i)
    artificial_jobs = list(zip(machine1, machine2))

    jobs_sorted = sorted(enumerate(artificial_jobs), key=lambda x: min(x[1]))
    U = [job for job in jobs_sorted if job[1][0] < job[1][1]]
    V = [job for job in jobs_sorted if job[1][0] >= job[1][1]]
    # Concatenate U in order and V in reverse order, extracting only job indices
    sequence = [job[0] for job in U] + [job[0] for job in reversed(V)]
    return sequence

In [22]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]
    
    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

In [23]:
def marouane_heuristique(processing_times):
    n_jobs,n_machines = processing_times.shape
    best_sequence = None
    best_makespan = float('inf')

    for k in range(1,n_machines - 1):  # Generate all artificial 2-machine problems
        weights_front = np.array( [n_machines - i for i in range(k) ])
        weights_back = np.array([i + 1 for i in range(k,n_machines)])
        AM1 = processing_times[:, :k].dot(weights_front)
        AM2 = processing_times[:, k:].dot(weights_back)

        # artificial_jobs = list(zip(AM1, AM2))
        sequence = johnsons_rule(AM1 , AM2)
        makespan = calculate_makespan(processing_times, sequence)
        if makespan < best_makespan:
            best_makespan = makespan
            best_sequence = sequence

    return best_sequence